In [1]:
import pickle
import pandas as pd

from ipynb.fs.full.createModel import getTrainingSet, createDataFrame
from ipynb.fs.full.configureCWD import setCurrentWorkingDirectory
import os.path
from os import path


In [2]:
def getTrainingSetCSV(startYear, startMonth, startDay, endYear, endMonth, endDay, season, startDateOfSeason, filename='gamesWithInfo.csv'):

    # Gets date, teams, and z-score difs for every game within range
    rangeOfGames = getTrainingSet(startYear, startMonth, startDay, endYear, endMonth, endDay, season, startDateOfSeason)
    rangeOfGamesDataframe = createDataFrame(rangeOfGames)
    
#     setCurrentWorkingDirectory('Data')

    rangeOfGamesDataframe.to_csv(filename)

In [3]:
# Creates a csv file that gives predictions for range of games
# Prints accuracy of model in predicting games for specified range
# gameDataFilename and outputFilename must be '.csv' files
def getPredictionsCSV(gameDataFilename, outputFilename):

#     setCurrentWorkingDirectory('Data')

    gamesWithZScoreDifs = pd.read_csv(gameDataFilename)

    withoutNums = gamesWithZScoreDifs.loc[:, 'Home':'Date']  # Slices dataframe to only includes home through date
    justZScoreDifs = gamesWithZScoreDifs.loc[:, 'W_PCT':'TS_PCT']  # Slices dataframe to only include statistical differences

#     setCurrentWorkingDirectory('SavedModels')
    with open('finalized_model.pkl', 'rb') as file:  # Change filename here if model is named differently
        pickleModel = pickle.load(file)

    predictions = pickleModel.predict(justZScoreDifs)  # Creates list of predicted winners and losers
    probPredictions = pickleModel.predict_proba(justZScoreDifs)  # Creates list of probabilities that home team wins

    numCorrect = 0
    numWrong = 0
    allGames = []

    for i in range(len(probPredictions)):

        winProbability = probPredictions[i][1]
        homeTeam = withoutNums.iloc[i, 0]
        awayTeam = withoutNums.iloc[i, 1]
        date = withoutNums.iloc[i, 10]

        currentGameWithPred = [date, homeTeam, awayTeam, winProbability]

        allGames.append(currentGameWithPred)

        # Creates dataframe that holds all games info and predictions
        predictionsDF = pd.DataFrame(
            allGames,
            columns=['Date', 'Home', 'Away', 'Home Team Win Probability']
        )

#         setCurrentWorkingDirectory('Data')
        predictionsDF.to_csv(outputFilename)  # Saves game info with predictions in data folder as csv file

        value = withoutNums.iloc[i,9]
        if value == predictions[i]:
            numCorrect += 1
        else :
            numWrong += 1

    print('Accuracy:')
    print((numCorrect)/(numCorrect+numWrong))  # Prints accuracy of model in predicting games for specified range

In [4]:
# # Generates probability predictions over specified range of games exports them to a csv with game info
# # gameDataFilename and outputFilename must end in '.csv'
# # season must be in form 'yyyy-yy' and startDateOfSeason must be in form 'mm/dd/yyyy'
def makePastPredictions(startYear, startMonth, startDay, endYear, endMonth, endDay, season, startDateOfSeason,
                       gameDataFilename='gamesWithInfo.csv'):

    # Obtains info for range of games
    getTrainingSetCSV(startYear, startMonth, startDay, endYear, endMonth, endDay, season, startDateOfSeason,
                          gameDataFilename)
#     # Makes probabilities for range of games
#     getPredictionsCSV(gameDataFilename, outputFilename)

In [ ]:
# # start date (yyyy, m, d) (must be at least three days after start of season), end date (yyyy, m, d) (non-inclusive),
# # season(yyyy-yy), start date of season (mm/dd/yyyy), .csv filename for games with z score differences,
# # .csv filename for games with predictions
# # EDIT THIS
makePastPredictions(2019, 10, 25, 2020, 3, 12, '2019-20', '10/16/2019',
                    'gamesWithInfo.csv')

In [ ]:
csv = pd.read_csv("gamesWithInfo.csv")
start_date = csv["Date"][len(csv)-1].split('/')
print(start_date)
month = int(start_date[0])
day = int(start_date[1])
print(month)
print(day)

In [5]:
def updateCSV(filename):
    old_df = pd.read_csv(filename)
    
    start_date = old_df["Date"][len(old_df)-1].split('/')
    month = int(start_date[0])
    day = int(start_date[1])
    year = int(start_date[2])
    
    new_df = createDataFrame(getTrainingSet(year, month, (day+1),2019, 10, 29,'2019-20', '10/16/2019'))
    uptodate = old_df.append(new_df)
    uptodate.drop(columns = ['Unnamed: 0'])
    
    uptodate.to_csv(filename)

In [6]:
def month_to_month(filename, season = '2018-19'):
    if (path.exists(filename) != True): #file doesnt exist
        getTrainingSetCSV(2018, 10, 25, 2018, 11, 1,'2018-19', '10/16/2018')

    if (path.exists(filename) == True): #file does exist
        old_df = pd.read_csv(filename)
        start_date = old_df["Date"][len(old_df)-1].split('/')
        month = int(start_date[0])
        day = int(start_date[1])
        year = int(start_date[2])
        
        new_month = (month%12)+1
        new_df = old_df
        
        while (new_month != 7):
            new_df = pd.read_csv(filename)
            new_df.append(createDataFrame(getTrainingSet(year, new_month,1,2019,(new_month%12)+1, 1,'2018-19', '10/16/2018')), ignore_text=True)
            new_df.to_csv(filename)
            new_month = (new_month+1)%12    

In [ ]:
month_to_month('gamesWithInfo.csv')

11/01/2018
['Philadelphia 76ers', 'LA Clippers', -0.29134871082327607, 1.1259214797773416, 2.0148873837911423, -1.191780164151846, -0.8328684684746059, -0.835604182564774, 0.3997919784781244, -1.508555799190075, 1]
['Boston Celtics', 'Milwaukee Bucks', -1.173601849231789, -1.095491169513091, -1.1102440686196091, -1.8227226039969415, -1.4158763964068304, -2.467021872334094, -0.39979197847812475, -2.771532747349211, 1]
['Cleveland Cavaliers', 'Denver Nuggets', -2.92990112011013, -1.2172124105701003, -1.3980851234469163, 0.6309424398450955, -2.082171171186515, -0.0795813507204529, 2.878502245042497, -0.07016538600884087, 0]
['Portland Trail Blazers', 'New Orleans Pelicans', 0.5868009246158944, 1.8866792363836564, -2.384968739997678, 0.9814660175368151, 0.832868468474606, -0.0795813507204528, -1.3792823257495277, 0.1754134650221023, 1]
['Atlanta Hawks', 'Sacramento Kings', -1.3910875066069104, -0.39559403343528393, 0.04112015068961575, 1.962932035073629, -1.0549667267345009, -1.79058039121

['Dallas Mavericks', 'Washington Wizards', 0.0, 1.0987531715941894, 1.1263649423622355, 1.519208964621812, 0.42235468108267504, 0.5048025732619169, -0.2615525537942094, 0.08051764321849439, 1]
['Charlotte Hornets', 'Atlanta Hawks', 0.7680667792645969, 1.3249670598635797, 0.6307643677228512, -3.383692693930401, 2.2947937672158667, 2.8911420105000643, -0.7846576613826313, 0.6843999673571977, 1]
11/07/2018
['Los Angeles Lakers', 'Minnesota Timberwolves', 0.16901470325627, 0.06505881788595991, 0.9677824501474468, 1.4463655575990082, 0.8419751406130103, 0.7406905910333036, -0.7753551739662673, 1.6222004108864232, 1]
['Orlando Magic', 'Detroit Pistons', -0.20657352620210756, -0.8457646325174973, 2.672922957550094, -0.8953691547041486, -0.84197514061301, -0.6958002521827996, 0.7503437167415468, -0.527215133538089, 0]
['Cleveland Cavaliers', 'Oklahoma City Thunder', -2.140852907912752, -0.683117587802594, -0.23042439289224947, 1.033118255427863, -1.8123871670822422, -0.20200652482726605, 3.001

['LA Clippers', 'Golden State Warriors', -1.3320411435043051, 0.1408293367899307, -3.5490065454656943, 0.0, -1.0171269974452293, -1.7523210487805714, -0.24634069563824573, -1.7387442403255995, 1]
['Miami Heat', 'Philadelphia 76ers', -0.7799784642572736, -0.10562200259244747, -1.0387336230631306, 0.29484247371966765, 0.14303348401573535, 0.4323909081406616, 0.27713328259302483, -0.7037774306079807, 0]
['Sacramento Kings', 'San Antonio Spurs', -0.49634993180008313, -1.1618420285169313, -0.3029639733934125, 1.769054842318012, -0.47677828005245115, -0.2958464108330827, 0.4003036304121499, 0.662378758219276, 1]
['Minnesota Timberwolves', 'Brooklyn Nets', -0.7799784642572739, -0.8801833549370699, -1.1252947583183897, -1.4742123685983433, -1.0171269974452293, -0.6144502378840968, 1.3548738260103537, -0.827973447774095, 1]
['Washington Wizards', 'Orlando Magic', -1.073736587159364, -2.1476473860464487, -1.9909061108709991, 0.36855309214958576, -0.6198117640681867, 0.5006631567944493, 2.1246884

['Dallas Mavericks', 'Golden State Warriors', -1.920878110522929, -0.38377582266806176, -2.7536807480768744, 1.9385446271792375, -1.1212032909521132, -2.1051350881415796, -0.6898932963298436, -1.7691447705629062, 1]
['Charlotte Hornets', 'Philadelphia 76ers', -0.526595868305351, -0.7675516453361235, -0.2800353303129013, -2.5350198970805438, 0.9254376369763473, 1.5073806803976746, 0.3449466481649218, -0.1516409803339578, 0]
['New Orleans Pelicans', 'Denver Nuggets', -0.8018618903740575, -0.5756637340020927, 0.0, 0.14911881747532554, -1.1567970462204342, -0.025989322075824273, 2.1731638834390115, 0.40437594755723005, 1]
11/18/2018
['San Antonio Spurs', 'Golden State Warriors', -1.221093038098119, 0.14969852714102228, -2.637252934342199, -2.0119819491421653, -1.393829714008172, -2.0289281135190333, -0.10105714310064623, -3.407122250141817, 1]
['Orlando Magic', 'New York Knicks', 1.481909026818106, -0.4865202132083145, 3.390753772725685, -1.160758816812788, 0.5645892512438166, 0.1979442061

['LA Clippers', 'Memphis Grizzlies', -0.33864561218614053, 1.6891455863682494, -1.140354302171639, 0.5325440572181881, 0.21893456504681652, 1.7538508783950029, 1.7735146360154965, 1.4675872542831285, 1]
['Detroit Pistons', 'Houston Rockets', -0.1721926841624441, 1.6891455863682494, -0.14254428777145356, -0.35502937147879265, 0.0, -0.51266410291546, -0.624014408968412, -1.4675872542831347, 1]
['Indiana Pacers', 'San Antonio Spurs', 0.8035658594247413, -1.1900798449412675, 0.47514762590484916, 2.396448257481849, 0.8210046189255619, -0.16189392723646026, -1.6421431814958285, 1.2982502634043054, 0]
['Atlanta Hawks', 'Boston Celtics', -1.9113387942031346, -1.8427042760380896, -0.522662388495333, 3.2840216861788303, -2.2440792917298693, -0.971363563418773, 2.6602719540232447, 0.11289132725254891, 0]
['Milwaukee Bucks', 'Phoenix Suns', 3.3807163657226615, 3.9157465865809415, 0.9978100144001838, -2.041418886003055, 4.141512188802279, 3.6426133628203905, -2.9887005903224058, 2.1449352177984213,

['Milwaukee Bucks', 'Chicago Bulls', 2.93307766862657, 2.9917281312211155, 2.1443833335370153, -0.27162771206382125, 3.7804598528362465, 4.101322974806301, -1.696040694851853, 3.3213720287771116, 1]
['Minnesota Timberwolves', 'San Antonio Spurs', -0.15236767109748428, -0.7180147514930666, -0.5127873188892867, 1.4486811310070518, -0.31019157766861505, -0.7584638378066451, -0.19953419939433342, -0.3940610881599971, 1]
['Charlotte Hornets', 'Atlanta Hawks', 1.6633470761475355, 0.9972427104070385, 0.5127873188892851, -4.164958251645276, 2.73356327820467, 3.1743116174870663, -0.6983696978801764, 1.3510665879771264, 1]
['Oklahoma City Thunder', 'Cleveland Cavaliers', 2.672782897168368, 0.8775735851581951, 0.5594043478792208, -1.1770534189432302, 2.3070498589103243, 0.19663877276468636, -3.691382688795206, -0.5066499704914247, 1]
['New Orleans Pelicans', 'Washington Wizards', 0.48249762514203287, 2.4731619218094565, 1.3052768117181832, 0.9054257068794074, 0.9693486802144223, 1.236015143092311